## Spike Fault Modell auf Spieler - Ebene

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import set_config
set_config(transform_output='pandas')
from sklearn.impute import SimpleImputer

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate,StratifiedKFold, cross_val_score, train_test_split
import os
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder,MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor

In [2]:
os.chdir('C:/Users/Katharina/Desktop/Weiterbildung/Bootcamp/Bootcamp/Final_project/data')
DatS = pd.read_csv('DatenML_V1.csv', sep=';')

In [3]:

def assign_team_label(row):
    # Lese den Teamnamen der aktuellen Zeile aus
    team_name = row['TeamName']
    # Zerlege den String in @TeamAName in einzelne Namen – Trimme Leerzeichen
    team_a_names = [name.strip() for name in row['@TeamAName'].split('/')]
    # Falls team_name in der Liste der TeamA-Namen enthalten ist, handelt es sich um Team A, ansonsten um Team B
    return 'A' if team_name in team_a_names else 'B'


In [4]:
DatS['TeamDesignation'] = DatS.apply(assign_team_label, axis=1)

In [6]:
DatS2 = DatS.dropna(subset=['temperature_2m', 'precipitation', 'wind_speed_10m', 'rain', 'wind_gusts_10m'])

In [8]:
df = DatS2.copy()
# Erstellen der neuen Spalte "Team1":
# Wenn TeamDesignation = A, dann soll TeamAName in Team1 stehen,
# andernfalls (also bei B) TeamBName.
df["Team1"] = np.where(
    df["TeamDesignation"].str.upper() == "A",
    df["@TeamAName"],
    df["@TeamBName"]
)

# Erstellen der Spalte "Team2" als das jeweils andere Team:
df["Team2"] = np.where(
    df["TeamDesignation"].str.upper() == "A",
    df["@TeamBName"],
    df["@TeamAName"]
)

In [31]:
#bestimmte Variablen entfernen
data1 = df.drop(['FirstName', 'LastName', 'TeamDesignation', '@PointsTeamASet3', '@PointsTeamBSet3', 
       'FederationCode_y', 'Gender_x','TeamName','ItemType','@NoInTournament', '@LocalDate',
       '@LocalTime','@Court','TeamFault','NoMatch','PointTotal','TournamentNo',
       '@LocalDate', '@LocalTime','ID_row', 'ID_DefaultCity','NoPlayer1', 'NoPlayer2', 'Name_y','team_idx',
        'SpikeTotal','@TeamAName', '@TeamBName',
        '@DurationSet3'], axis=1)


In [32]:
y = data1.pop('SpikeFault')
X = data1.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

In [41]:
X.head()

,SpikePoint,ServeFault,ServePoint,ServeTotal,BlockPoint,BlockTotal,DigTotal,ReceptionFault,@PointsTeamASet1,@PointsTeamBSet1,...,@DurationSet1,@DurationSet2,temperature_2m,precipitation,wind_speed_10m,rain,wind_gusts_10m,TeamFault_team,Team1,Team2
0,10,3,3,28,0,0,16,1,21.0,19.0,...,1078.0,1084.0,31.0,0.0,19.3,0.0,42.1,22.0,Ana Patrícia/Duda,Tina/Anastasija
1,18,3,4,27,1,4,5,0,21.0,19.0,...,1078.0,1084.0,31.0,0.0,19.3,0.0,42.1,23.0,Tina/Anastasija,Ana Patrícia/Duda
2,14,6,0,21,0,1,18,3,21.0,19.0,...,1078.0,1084.0,31.0,0.0,19.3,0.0,42.1,23.0,Tina/Anastasija,Ana Patrícia/Duda
3,20,1,0,25,2,9,10,3,21.0,19.0,...,1078.0,1084.0,31.0,0.0,19.3,0.0,42.1,22.0,Ana Patrícia/Duda,Tina/Anastasija
4,14,3,3,24,0,3,4,1,17.0,21.0,...,1177.0,1042.0,31.0,0.0,19.3,0.0,42.1,17.0,Talita/Thamela,Anouk/Mäder


In [33]:
# select categorical and numerical column names
nominal_features = X.select_dtypes(include=['object']).columns.tolist()
# Define feature types
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Create preprocessors for different feature types
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler())
])

nominal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Build the column transformer
transformers = [
    ('num', numeric_transformer, numeric_features),
    ('nom', nominal_transformer, nominal_features)
]

preprocessor = ColumnTransformer(transformers=transformers)

In [36]:
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error

model_scores = []
def score_model(model_name, y_true, y_pred):
  scores = {
      'Model': model_name,
      'MAE ($)': round(mean_absolute_error(y_true, y_pred), 2),
      'RMSE ($)': round(root_mean_squared_error(y_true, y_pred), 2),
      'MAPE (%)': round(100 * mean_absolute_percentage_error(y_true, y_pred), 2),
      'R-Squared': round(r2_score(y_true, y_pred), 3)
  }
  return scores

In [37]:
from sklearn.ensemble import GradientBoostingRegressor
gbr_pipeline = make_pipeline(preprocessor,
                             RobustScaler(),
                             GradientBoostingRegressor())

gbr_pipeline.fit(X_train, y_train)

gbr_predictions = gbr_pipeline.predict(X_test)

model_scores.append(score_model('GradientBoostigRegressor', y_test, gbr_predictions))
pd.DataFrame(model_scores)

,Model,MAE ($),RMSE ($),MAPE (%),R-Squared
0,GradientBoostigRegressor,1.82,2.32,9.111297e+16,0.186


In [40]:
from sklearn.ensemble import RandomForestRegressor
rf_pipeline = make_pipeline(preprocessor,
                             RandomForestRegressor())

rf_pipeline.fit(X_train, y_train)

rf_predictions = rf_pipeline.predict(X_test)

model_scores.append(score_model('RandomForest', y_test, rf_predictions))
pd.DataFrame(model_scores)

,Model,MAE ($),RMSE ($),MAPE (%),R-Squared
0,GradientBoostigRegressor,1.82,2.32,9.111297e+16,0.186
1,RandomForest,1.79,2.31,7.512124e+16,0.195
2,RandomForest,1.79,2.32,7.591767e+16,0.192
